In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mesolitica-storage.json'

In [2]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-general')

In [ ]:
!rm t5-base-summary/*

model = '537600'

blob = bucket.blob(f't5-base-generator-v1/model.ckpt-{model}.data-00000-of-00002')
blob.download_to_filename(f't5-base-summary/model.ckpt-{model}.data-00000-of-00002')

blob = bucket.blob(f't5-base-generator-v1/model.ckpt-{model}.data-00001-of-00002')
blob.download_to_filename(f't5-base-summary/model.ckpt-{model}.data-00001-of-00002')

blob = bucket.blob(f't5-base-generator-v1/model.ckpt-{model}.index')
blob.download_to_filename(f't5-base-summary/model.ckpt-{model}.index')

blob = bucket.blob(f't5-base-generator-v1/model.ckpt-{model}.meta')
blob.download_to_filename(f't5-base-summary/model.ckpt-{model}.meta')

blob = bucket.blob('t5-base-generator-v1/checkpoint')
blob.download_to_filename('t5-base-summary/checkpoint')

blob = bucket.blob('t5-base-generator-v1/operative_config.gin')
blob.download_to_filename('t5-base-summary/operative_config.gin')

with open('t5-base-summary/checkpoint', 'w') as fopen:
    fopen.write(f'model_checkpoint_path: "model.ckpt-{model}"')

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import t5

In [5]:
model = t5.models.MtfModel(
    model_dir='t5-base-summary',
    tpu=None,
    tpu_topology=None,
    model_parallelism=2,
    batch_size=1,
    sequence_length={"inputs": 1024, "targets": 1024},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=3,
    iterations_per_loop=100,
    mesh_shape="model:1,batch:1", 
    mesh_devices=["cpu:0"]
)

In [6]:
string = '1. Dr M perlu dikekalkan sebagai perdana menteri. 2. Muhyiddin perlulah menolong Dr M. 3. rakyat perlu menolong Muhyiddin.'

In [ ]:
string = '1. kerajaan perlu tolong gotong royong. 2. masyarakat juga perlu menolong kerajaan. 3. ibu bapa perlu memastikan anak menolong kerajaan'

In [ ]:
string = '1. Neelofa tetap dengan keputusan untuk berkahwin akhir tahun ini. 2. Long Tiger sanggup membantu Neelofa. 3. Tiba-tiba Long Tiger bergaduh dengan Husein.'

In [ ]:
string = '1. menambahkan ilmu tentang tempat yang dilawati. 2. memanfaatkan masa yang terluang ke tempat yang berfaedah. 3. menambahkan semangat kecintaan negara. 4. memberikan hiburan - sambil melawat sambil berhibur dan berseronok'

In [7]:
print(string)
with tf.io.gfile.GFile('test.txt', "w") as f:
    f.write("karangan: %s\n" % string)

1. Dr M perlu dikekalkan sebagai perdana menteri. 2. Muhyiddin perlulah menolong Dr M. 3. rakyat perlu menolong Muhyiddin.


In [8]:
model.predict(
    input_file='test.txt',
    output_file='out.txt',
    temperature=0.7,
    beam_size=1,
    sentencepiece_model_path='sp10m.cased.t5.model'
)

INFO:tensorflow:Using config: {'_model_dir': 't5-base-summary', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': None, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, per_host_input_for_training=4, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training

In [16]:
!rm -rf output/*

In [17]:
model.batch_size = 1
saved_model_path = model.export(
    'output',
    checkpoint_step=-1,
    beam_size=1,
    temperature=0.8,
    sentencepiece_model_path='sp10m.cased.t5.model'
)

INFO:tensorflow:Using config: {'_model_dir': 't5-base-summary', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': None, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, per_host_input_for_training=4, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training

In [18]:
saved_model_path.decode()

'output/1590369228'

In [19]:
import tensorflow_text  

tf.compat.v1.reset_default_graph()
sess = tf.InteractiveSession()
meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], saved_model_path.decode())
signature_def = meta_graph_def.signature_def["serving_default"]
pred = lambda x: sess.run(
    fetches=signature_def.outputs["outputs"].name, 
    feed_dict={signature_def.inputs["input"].name: x}
)

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:Restoring parameters from output/1590369228/variables/variables


In [20]:
q = f'karangan: {string}'

pred([q])[0].decode('utf-8')

': Presiden Bersatu, Tan Sri Muhyiddin Yassin perlu mengekalkan Tun Dr Mahathir Mohamad sebagai perdana menteri berbanding Datuk Seri Anwar Ibrahim yang hanya minta bantuan untuk menyelesaikan kemelut kedudukan negara.Muhyiddin berkata, ini kerana semua pihak tahu masalah yang dihadapi oleh Perdana Menteri adalah di luar bidang kuasa beliau sendiri.Katanya, Muhyiddin perlu membantu beliau kerana beliau percaya rakyat Malaysia tahu apa yang berlaku di luar bidang kuasa beliau."Apa yang berlaku di luar bidang kuasa Dr Mahathir... semua tahu bahawa ini berlaku di bawah kepimpinan Anwar."Muhyiddin dan seluruh rakyat yang tahu apa yang berlaku di Johor."Ini kerana di Johor ini, majoriti menteri-menteri dalam Pakatan Harapan banyak sangat ketua-ketua parti."Jadi Muhyiddin perlu bantu Dr Mahathir sebab rakyat tahu apa yang berlaku di Johor Bahru," katanya dalam satu kenyataan di sini, pada Jumaat.Dalam pada itu, Muhyiddin berkata, rakyat juga perlu menolong Muhyiddin untuk menyelesaikan masal

In [21]:
string = '1. kerajaan perlu tolong gotong royong. 2. masyarakat juga perlu menolong kerajaan. 3. ibu bapa perlu memastikan anak menolong kerajaan'
q = f'karangan: {string}'

pred([q])[0].decode('utf-8')

'Shah Alam - Masyarakat juga perlu membantu kerajaan dalam menangani masalah politik dan rasuah dalam negara. Menurut Timbalan Menteri Perumahan dan Kerajaan Tempatan, Datuk Raja Kamarul Bahrin Shah Raja Ahmad, pengurusan kerajaan sepatutnya melibatkan diri dalam menyelesaikan masalah yang dibangkitkan oleh masyarakat. Katanya, semua pihak yang terlibat harus memastikan kerja-kerja pentadbiran dilaksanakan mengikut undang-undang yang ditetapkan. "Kalau kita lihat dalam beberapa hari ini banyak unsur-unsur politik yang melibatkan salah guna kuasa, rasuah dan salah guna kuasa berlaku dalam negara kita. "Ini kerana masalah rasuah menjadi antara isu politik yang masih berlaku ketika ini. "Oleh itu, kita kena bantu. Sudah tentu kajian dan perancangan perlu dilakukan untuk selesaikan masalah ini," katanya kepada Sinar Harian. Beliau berkata demikian sebagai mengulas kenyataan Menteri Kewangan, Lim Guan Eng baru-baru ini yang meminta kerajaan meringankan beban ditanggung rakyat akibat kegiata

In [22]:
string = '1. Neelofa tetap dengan keputusan untuk berkahwin akhir tahun ini. 2. Long Tiger sanggup membantu Neelofa. 3. Tiba-tiba Long Tiger bergaduh dengan Husein.'
q = f'ringkasan: {string}'

pred([q])[0].decode('utf-8')

'Shah Alam: Pelakon, Neelofa tetap dengan keputusan untuk berkahwin akhir tahun ini dengan berkahwin dengan Long Tiger yang menjadi teman baiknya. Neelofa berkata, dia tidak akan berputus asa kerana tiada apa yang perlu dibimbangkan. "Tak apalah, saya yang akan uruskan semuanya. Tapi biarlah proses itu berjalan dengan baik. Selagi tak selesai, saya tak akan melarikan diri. "Tapi sebagai suami, saya tetap dengan keputusan untuk berkahwin. Tapi saya tak akan kecewa kerana tiada apa yang perlu dibimbangkan. "Jika saya berkahwin dengan Long Tiger, antarabangsa yang akan menahan gosip seumpama ini, saya tidak akan halang. "Sebab itu saya tidak mahu timbul isu yang boleh mengganggu perkahwinan," katanya kepada Bh Online. Baru-Baru ini, kecoh apabila Long Tiger bergaduh dengan Datuk Husein Awang (Datuk Afdlin Shauki). Dalam perbincangan dengan suami, pelakon itu mendedahkan dia sudah meminta kebenaran supaya dia berkahwin dengan gadis pilihan di Singapura iaitu Hairul Azreen. Bagaimanapun, pe

In [23]:
!rm -rf model

In [24]:
os.system(f'mv {saved_model_path.decode()} model')

0

In [25]:
!tar -czvf sample-generator-t5-base.tar.gz model

model/
model/saved_model.pb
model/variables/
model/variables/variables.data-00000-of-00002
model/variables/variables.data-00001-of-00002
model/variables/variables.index


In [26]:
import boto3

bucketName = 'huseinhouse-storage'
Key = 'sample-generator-t5-base.tar.gz'
outPutname = "v35/generator/sample-generator-t5-base.tar.gz"

s3 = boto3.client('s3')
s3.upload_file(Key,bucketName,outPutname)